In [104]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.55 s (started: 2023-02-24 13:34:59 +00:00)


In [105]:
!pip install Faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 3.85 s (started: 2023-02-24 13:35:32 +00:00)


In [106]:
import pandas as pd
import numpy as np
import random

time: 731 µs (started: 2023-02-24 13:35:36 +00:00)


# How to Benchmark
#### Give a man a fish and you feed him for a day, teach a man to fish and you feed him for a lifetime

## timeit ⏰

In [107]:
def count_multiples(n):
  count = 0
  for i in range(n):
    if i % 10 == 0:
      count = count + 1
  return count

time: 1.11 ms (started: 2023-02-24 13:37:29 +00:00)


In [108]:
%%timeit
count_multiples(100)

7.85 µs ± 221 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
time: 6.66 s (started: 2023-02-24 13:37:35 +00:00)


## Line Profiling 🧙

In [109]:
#https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.07-Timing-and-Profiling.ipynb
!pip install line_profiler
%load_ext line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
time: 3.55 s (started: 2023-02-24 13:38:16 +00:00)


In [110]:
%lprun -f count_multiples count_multiples(100)

time: 3.57 ms (started: 2023-02-24 13:38:54 +00:00)


# Section 1 - Filtering

In [62]:
# https://towardsdatascience.com/five-killer-optimization-techniques-every-pandas-user-should-know-266662bd1163
# def create_employees(n=4_096_000):
#     city_list = [
#         "New York", "Manchester", "California", "Munich", "Bombay",
#         "Sydeny", "London", "Moscow", "Dubai", "Tokyo"
#     ]

#     job_list = [
#         "Software Development Engineer", "Research Engineer", 
#         "Test Engineer", "Software Development Engineer-II", 
#         "Python Developer", "Back End Developer", 
#         "Front End Developer", "Data Scientist", 
#         "IOS Developer", "Android Developer"
#     ]

#     cmp_list = [
#         "Amazon", "Google", "Infosys", "Mastercard", "Microsoft", 
#         "Uber", "IBM", "Apple", "Wipro", "Cognizant"
#     ]

#     data = []
#     for i in range(n):    
#         company = random.choice(cmp_list)
#         job = random.choice(job_list)
#         city = random.choice(city_list)
#         salary = int(round(np.random.rand(), 3)*10**6)
#         employment = random.choices(["Full Time", "Intern"], weights=(80, 20))[0]
#         rating = round((np.random.rand()*5), 1)
        
#         data.append([i, company, job, city, salary, employment, rating])
        
#     data = pd.DataFrame(data, columns=[
#         "Id", "Company Name", "Employee Job Title",
#         "Employee Work Location", "Employee Salary", 
#         "Employment Status", "Employee Rating"
#         ]
#     )
#     return data
# employees = create_employees(n=800000)

time: 424 µs (started: 2023-02-24 13:03:04 +00:00)


In [111]:
employees = pd.read_csv("employees.csv")

time: 563 ms (started: 2023-02-24 13:41:26 +00:00)


In [112]:
employees.sample(5)

,Id,Company Name,Employee Job Title,Employee Work Location,Employee Salary,Employment Status,Employee Rating
268992,268992,IBM,Software Development Engineer-II,Manchester,649000,Full Time,3.3
749259,749259,Amazon,Software Development Engineer,London,728000,Full Time,0.6
35260,35260,Cognizant,Data Scientist,Munich,32000,Intern,2.4
668573,668573,Mastercard,Back End Developer,Sydeny,373000,Full Time,0.0
306845,306845,Microsoft,Software Development Engineer-II,New York,592000,Full Time,1.9


time: 36.4 ms (started: 2023-02-24 13:41:30 +00:00)


In [113]:
%%timeit
employees[employees["Company Name"] == "Amazon"]

77.2 ms ± 22.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
time: 6.13 s (started: 2023-02-24 13:42:22 +00:00)


In [114]:
data_grp = employees.groupby("Company Name")

time: 881 µs (started: 2023-02-24 13:42:42 +00:00)


In [115]:
%%timeit
data_grp.get_group("Amazon")

7.78 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
time: 6.3 s (started: 2023-02-24 13:43:00 +00:00)


## Summary
For filtering of your DataFrame on categorical data, prefer grouping the data first using the `groupby()` method. After that, fetch the desired groups using the `get_group()` method

* Only applicable to filtering based on categorical data

# Section 2 - Merge - Index Optimization
## Merge vs join
Both join and merge can be used to combines two dataframes but the join method combines two dataframes on the basis of their indexes whereas the merge method is more versatile and allows us to specify columns beside the index to join on for both dataframes.

In [68]:
# from faker import Faker
# fake = Faker()
#
# def create_people(n=100):
#   data = []
#   for i in range(n):
#     person_name = fake.name()
#     address = fake.address()
#     age = random.randint(18, 65)
#     data.append([i, person_name, address, age])
#
#   data = pd.DataFrame(data, columns=[
#         "Id", "Full Name", "Address", "Age"
#         ]
#   )
#   return data
# people = create_people(n=800000)
# employees = create_employees(n=800000)

time: 541 µs (started: 2023-02-24 13:03:15 +00:00)


In [116]:
employees = pd.read_csv("employees.csv")
people = pd.read_csv("people.csv")

time: 2.64 s (started: 2023-02-24 13:46:06 +00:00)


In [117]:
people.sample(5)

,Id,Full Name,Address,Age
403021,403021,Gina Murphy MD,"PSC 6806, Box 2779\nAPO AE 74469",49
93046,93046,Ruth Banks,"11715 Adam Summit\nFryeport, WI 11036",38
437897,437897,Becky Morales,"164 Ryan Island\nLake Jasonberg, VT 57530",53
747369,747369,Rhonda Vargas,"109 Steven Glen\nTurnermouth, NY 03130",36
729708,729708,Rachel Short,"9594 Hughes Circles\nPearsonstad, GU 15899",45


time: 34.7 ms (started: 2023-02-24 13:46:11 +00:00)


In [118]:
%%timeit
merged = pd.merge(employees, people, on="Id")

294 ms ± 5.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
time: 2.5 s (started: 2023-02-24 13:46:45 +00:00)


In [119]:
employees_df2 = employees.set_index("Id")
people_df2 = people.set_index("Id")

time: 38.3 ms (started: 2023-02-24 13:46:59 +00:00)


In [120]:
%%timeit
merged_v2 = pd.merge(employees_df2, people_df2, on="Id")

236 ms ± 36.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
time: 2.04 s (started: 2023-02-24 13:47:17 +00:00)


In [121]:
%%timeit
merged_v3 = employees_df2.join(people_df2, on="Id")

166 ms ± 22.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
time: 13.4 s (started: 2023-02-24 13:47:51 +00:00)


## Summary
For DataFrame joins, always change the index of both the DataFrames and set it to the column(s) you want to execute the join condition on.

# Section 3 - Filter before Join

In [122]:
employees = pd.read_csv("employees.csv")
people = pd.read_csv("people.csv")

time: 2.3 s (started: 2023-02-24 13:49:10 +00:00)


In [76]:
employees.sample(5)

,Id,Company Name,Employee Job Title,Employee Work Location,Employee Salary,Employment Status,Employee Rating
290424,290424,IBM,IOS Developer,New York,695000,Full Time,0.5
458263,458263,Amazon,Android Developer,Manchester,861000,Full Time,2.3
737310,737310,IBM,Software Development Engineer-II,Sydeny,649000,Full Time,3.8
643026,643026,Mastercard,Research Engineer,Munich,875000,Full Time,4.9
214728,214728,Microsoft,IOS Developer,Moscow,972000,Full Time,4.2


time: 115 ms (started: 2023-02-24 13:03:31 +00:00)


In [123]:
%%timeit
result = people.merge(employees, on="Id", how="inner").query("Age > 35")

519 ms ± 13.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
time: 4.28 s (started: 2023-02-24 13:49:28 +00:00)


In [124]:
%%timeit
result_v2 = people.query("Age > 35").merge(employees, on="Id", how="inner")

420 ms ± 198 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
time: 3.37 s (started: 2023-02-24 13:49:46 +00:00)


## Filter before join to reduce the execution time

# Section 4 - Memory Optimization - use `inplace` parameter

In [125]:
# The “inplace” parameter allows us to modify a Pandas data frame in place, without creating a new one. By using this parameter,
# we can reduce memory usage and improve performance
# Modify the data frame in place
employees = pd.read_csv("employees.csv")

time: 505 ms (started: 2023-02-24 13:50:54 +00:00)


In [126]:
employees.columns

Index(['Id', 'Company Name', 'Employee Job Title', 'Employee Work Location',
       'Employee Salary', 'Employment Status', 'Employee Rating'],
      dtype='object')

time: 3.24 ms (started: 2023-02-24 13:51:14 +00:00)


In [127]:
employees.drop("Employee Rating", axis=1, inplace=True)

time: 33 ms (started: 2023-02-24 13:51:25 +00:00)


In [128]:
employees.columns

Index(['Id', 'Company Name', 'Employee Job Title', 'Employee Work Location',
       'Employee Salary', 'Employment Status'],
      dtype='object')

time: 4.06 ms (started: 2023-02-24 13:51:32 +00:00)


# Section 5 - DataFrame Processing

In [129]:
# https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam
games_df = pd.read_csv("games.csv")

time: 69 ms (started: 2023-02-24 13:52:15 +00:00)


In [130]:
games_df.sample(5)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
116,520720,Dear Esther: Landmark Edition,2017-02-14,True,True,False,Mostly Positive,78,1591,9.99,9.99,0.0,True
12924,953580,Tourist Bus Simulator,2018-12-06,True,False,False,Mixed,65,1975,29.99,29.99,0.0,True
4868,1004960,DiRT Rally 2.0 - Opel Manta 400,2019-04-08,True,False,False,Positive,84,13,1.99,1.99,0.0,True
10437,545980,Voodoo Vince: Remastered,2017-04-17,True,False,False,Very Positive,92,420,14.99,14.99,0.0,True
14413,1305130,The Lost Legends of Redwall™: The Scout Act 3,2021-12-24,True,True,False,Positive,100,10,9.99,9.99,0.0,True


time: 24.9 ms (started: 2023-02-24 13:52:18 +00:00)


In [131]:
games_df["date_release"] = games_df["date_release"].apply(pd.to_datetime)

time: 2.71 s (started: 2023-02-24 13:52:50 +00:00)


In [132]:
games_df.dtypes

app_id                     int64
title                     object
date_release      datetime64[ns]
win                         bool
mac                         bool
linux                       bool
rating                    object
positive_ratio             int64
user_reviews               int64
price_final              float64
price_original           float64
discount                 float64
steam_deck                  bool
dtype: object

time: 5.32 ms (started: 2023-02-24 13:52:57 +00:00)


## Use iterrows 🐢
#### This method returns the index as well as the row of a pandas dataframe

In [133]:
# Copy the DataFrame to keep the original DataFrame unchanged
temp_df = games_df.copy()
# List to store the computing results.  
new_col = []

time: 2.43 ms (started: 2023-02-24 13:53:23 +00:00)


In [134]:
# iterrows
current = np.datetime64("today")
for i, row in temp_df.iterrows():
    new_col.append(current - row["date_release"])
temp_df["date_release_iterrows"] = pd.Series(new_col)

time: 1.97 s (started: 2023-02-24 13:53:50 +00:00)


In [135]:
temp_df.sample(5)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck,date_release_iterrows
17980,201290,Sins of a Solar Empire: Trinity®,2011-11-16,True,False,False,Mostly Positive,78,625,19.99,19.99,0.0,True,4118 days
17606,1051950,Voyage,2021-02-18,True,False,False,Very Positive,85,88,14.99,14.99,0.0,True,736 days
12601,1443048,Forza Horizon 4: 2019 Chevrolet Corvette ZR1,2021-03-09,True,False,False,Mostly Positive,75,12,2.99,2.99,0.0,True,717 days
6910,1956370,WTF Do You Know?,2022-05-02,True,False,False,Very Positive,97,108,1.99,1.99,0.0,True,298 days
16852,446380,Sophie's Curse,2016-03-16,True,False,False,Very Positive,85,348,4.99,4.99,0.0,True,2536 days


time: 23.7 ms (started: 2023-02-24 13:54:12 +00:00)


## Use itertuples
#### This method is different from iterrows ithis method is faster as well as it also preserve the data type of a column

In [136]:
# Copy the DataFrame to keep the original DataFrame unchanged
temp_df = games_df.copy()
# List to store the computing results.  
new_col = []

time: 2.63 ms (started: 2023-02-24 13:54:33 +00:00)


In [137]:
# itertuples - This method is different from iterrows ithis method is faster
# as well as it also preserve the data type of a column
current = np.datetime64("today")
for row in temp_df.itertuples():
  new_col.append(current - row.date_release)
temp_df["date_release_itertuples"] = pd.Series(new_col)

time: 686 ms (started: 2023-02-24 13:54:40 +00:00)


In [138]:
games_df.shape

(21236, 13)

time: 5.4 ms (started: 2023-02-24 13:55:20 +00:00)


## Use pandas apply 🐎

In [139]:
# use apply
from datetime import datetime

def days_diff(release_date):
  current = np.datetime64("today")
  days_diff = current - release_date
  return days_diff

time: 1.02 ms (started: 2023-02-24 13:56:10 +00:00)


### apply for column

In [140]:
# apply column
%%timeit
games_df["days_diff_apply"] = games_df["date_release"].apply(days_diff)

487 ms ± 176 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
time: 3.77 s (started: 2023-02-24 13:56:29 +00:00)


In [141]:
%lprun -f days_diff games_df["date_release"].apply(days_diff)

time: 446 ms (started: 2023-02-24 13:56:52 +00:00)


### apply for dataframe

In [95]:
# apply dataframe
%%timeit
games_df["days_diff_apply_df"] = games_df.apply(lambda row: days_diff(row["date_release"]), axis=1)

1.24 s ± 356 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
time: 9.55 s (started: 2023-02-24 13:03:58 +00:00)


In [96]:
%lprun -f days_diff games_df.apply(lambda row: days_diff(row["date_release"]), axis=1)

time: 1.27 s (started: 2023-02-24 13:04:08 +00:00)


## Pandas Vectorize ⚡

In [97]:
# vectorize -  the ability to run an operation across a whole Series or DataFrame
# https://www.youtube.com/watch?v=nxWginnBklU
# https://www.youtube.com/watch?v=HN5d490_KKk
# https://github.com/s-heisler/pycon2017-optimizing-pandas
# https://medium.com/analytics-vidhya/understanding-vectorization-in-numpy-and-pandas-188b6ebc5398

time: 445 µs (started: 2023-02-24 13:04:09 +00:00)


In [142]:
%%timeit
# pandas vectorization
games_df["days_diff_vect"] = days_diff(games_df["date_release"])

628 µs ± 9.03 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
time: 5.12 s (started: 2023-02-24 13:59:52 +00:00)


In [143]:
%lprun -f days_diff days_diff(games_df["date_release"])

time: 4.96 ms (started: 2023-02-24 14:02:00 +00:00)


## NumPy Vectorize ⚡⚡

In [144]:
%%timeit
# numpy vectorization
games_df["days_diff_numpy"] = days_diff(games_df["date_release"].values)

407 µs ± 66.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
time: 3.44 s (started: 2023-02-24 14:03:01 +00:00)


In [145]:
%lprun -f days_diff days_diff(games_df["date_release"].values)

time: 2.44 ms (started: 2023-02-24 14:03:13 +00:00)


## Vectorize function
### Not all fucntions are vectorizable, e.g. aggregations aren't

In [146]:
# .vectorize function
vect_date_diff = np.vectorize(days_diff)

time: 579 µs (started: 2023-02-24 14:04:47 +00:00)


In [147]:
%%timeit
games_df["days_diff_numpy_vect"] = vect_date_diff(games_df["date_release"])

298 ms ± 7.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
time: 2.4 s (started: 2023-02-24 14:04:58 +00:00)


## Pandas Alternatives
* modin (https://github.com/modin-project/modin)
* polars (https://github.com/pola-rs/polars)
* numba (https://pandas.pydata.org/docs/user_guide/enhancingperf.html#numba-jit-compilation)
* dask (https://github.com/dask/dask)
* Apache Spark (https://github.com/apache/spark)

## Summary
* Avoid loops, if you can
* If you must loop, use apply, not iteration functions
* Vectorization is usually better than scalar operations
* Vector operations on NumPy arrays are more efficient than on native Pandas series
* Cython